In [1]:
import pandas as pd
import numpy  as np
from config import getClient
from checkMissing import checkMissing
from os import listdir
from os.path import isfile, join
import time

In [2]:
dataloc = getClient("OutFront")

In [3]:
panels = [f for f in listdir(dataloc+"panels/") if isfile(join(dataloc+"panels/", f))]

print("{:,.0f} panels to process".format(len(panels)))

699 panels to process


In [4]:
# Check each panel for missing dates
def processPanel(df):
    nan = df["population"].isnull().sum()
    if nan > 0:
        print("missing population: ", df["panel"].unique())
    missing = checkMissing(df, "day")
    if len(missing) > 0:
        return True
    else:
        df.set_index(["panel", "date"], inplace=True)
        grp = df.groupby(level=[0,1])
        for key, val in grp:
            missing = checkMissing(df, "hour")
            if len(missing) > 0:
                return True
        return False

In [5]:
count = 0
start = time.time()

for panel in panels:
    df = pd.read_csv(dataloc+"panels/"+panel,\
                     dtype={"hour":'int8',\
                            "population":'int32'})
    
    missing = processPanel(df)
    if missing:
        count += 1
        print("Missing: {}".format(panel))
    
print("Number missing dates: {}".format(count))
    
end = time.time()
print("Completed after {:.0f} minutes".format((end-start)/60))

Number missing dates: 0
Completed after 4 minutes
